In [1]:
%load_ext autoreload
%autoreload 3
%reload_ext autoreload
import pandas as pd
import openpyxl
import re
from datetime import datetime
from pathlib import Path
import os
from io import BytesIO
from azure.storage.blob import BlobServiceClient
from planification.pool import *

import numpy as np
from datetime import datetime, timedelta

In [2]:
df = read_archived_pool_proj()

In [3]:
df

,pool_slot,changeout_week,equipo,component_serial,changeout_date,arrival_week,pool_changeout_type,arrival_date,component_code
0,1,2023-W27,856,#EG08090003,2023-07-03,2023-W40,P,2023-10-02,bp
1,2,2023-W27,881,#EN12040115,2023-07-03,2023-W34,P,2023-08-21,bp
2,3,2023-W27,875,#EN11110073,2023-07-03,2023-W38,P,2023-09-18,bp
3,4,2023-W28,883,#EN12090007,2023-07-10,2023-W38,P,2023-09-18,bp
4,6,2023-W29,883,#EE06050200-1,2023-07-17,2023-W42,P,2023-10-16,bp
5,5,2023-W29,870,#EE14010117,2023-07-17,2023-W42,P,2023-10-16,bp
6,7,2023-W31,399,#EE14091188,2023-07-31,2023-W45,P,2023-11-06,bp
7,8,2023-W32,858,#EE14020272,2023-08-07,2023-W45,P,2023-11-06,bp
8,9,2023-W33,860,#EE13101143,2023-08-14,2023-W47,P,2023-11-20,bp
9,11,2023-W34,882,#EE14080990,2023-08-21,2023-W48,P,2023-11-27,bp


In [3]:
blob_service_client = BlobServiceClient(
    account_url=os.environ["AZURE_ACCOUNT_URL"],
    credential=os.environ["AZURE_SAS_TOKEN"],
)
blob_client = blob_service_client.get_blob_client(
    container=os.environ["AZURE_CONTAINER_NAME"],
    blob=f"{os.environ['AZURE_PREFIX']}/PLANIFICACION/POOL/2023.01 Pool Componentes MEL Planificadores.xlsx",
)
blob_data = blob_client.download_blob()
blob_data = BytesIO(blob_data.readall())

In [297]:
wb = openpyxl.load_workbook(blob_data, data_only=True)

sheet = wb["PROYECCIÓN"]

start_row = 20  # Starting from row 2
end_row = 33  # Ending at row 10
start_column = "F"  # Starting from column B
end_column = "EF"  # Ending at column E

# Create a DataFrame from the Excel data
sheet = sheet[f"{start_column}{start_row}:{end_column}{end_row}"]
data = []  # .iter_rows(min_row=20, values_only=True)
for row in sheet:
    data.append([cell.value for cell in row])

df = pd.DataFrame(data, columns=[cell.value for cell in sheet[1]])
df.set_index(df.columns[0], inplace=True)
df = df.iloc[1:]
# Generate date range
date_range = pd.date_range(start="2023-07-03", end="2025-12-31", freq="W-MON")

# Create the list of week strings
week_list = [f"{date.isocalendar()[0]}-W{date.isocalendar()[1]}" for date in date_range]

df.columns = week_list[:-1]
# df = df.dropna(how="all", axis=1)

df

,2023-W27,2023-W28,2023-W29,2023-W30,2023-W31,2023-W32,2023-W33,2023-W34,2023-W35,2023-W36,...,2025-W43,2025-W44,2025-W45,2025-W46,2025-W47,2025-W48,2025-W49,2025-W50,2025-W51,2025-W52
N°1,,,,,,,,,,,,,,,,,,,,,
N°1,1,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
N°2,1,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
N°3,1,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
N°4,None,1,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
N°5,None,None,1,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
N°6,None,None,1,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
N°7,None,None,None,None,1,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
N°8,None,None,None,None,None,1,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
N°9,None,None,None,None,None,None,1,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [298]:
from planification.pool.utils import (
    extract_info,
    idx_to_pool_slot,
    get_weeks_and_comments,
    get_end_week,
)


def get_weeks_and_comments(row, sheet):
    weeks = []
    comments = []
    row_idx = int(row.name.strip("N°"))
    col_idx = 1

    for col, value in row.items():
        # print(row_idx)
        if value == 1:
            weeks.append(col)
            cell = sheet[row_idx][col_idx]
            comments.append(cell.comment.text if cell.comment else "")
        col_idx += 1
    return pd.Series({"weeks": weeks, "comments": comments})


def idx_to_pool_slot(df):
    df = (
        df.rename_axis("pool_slot")
        .reset_index()
        .assign(pool_slot=lambda x: x["pool_slot"].str.strip("N°"))
    )
    return df


def extract_info(comment):
    equipo_match = re.search(r"Equipo:\s*(\d+)", comment)
    ns_match = re.search(r"NS:\s*(#+\w*-?\w*)", comment)
    ns = ns_match.group(1) if ns_match else None
    if ns:
        ns = re.sub(r"^#+", "#", ns)
    equipo = equipo_match.group(1) if equipo_match else None

    return equipo, ns


def get_end_week(row, sheet):
    weeks = []
    pool_changeout_types = []
    row_idx = int(row.name.strip("N°"))
    for col_idx in range(0, row.__len__()):
        if col_idx > 2:
            prev_cell = sheet[row_idx][col_idx - 1]
            cell = sheet[row_idx][col_idx]
            if (
                (prev_cell.fill.fgColor.rgb != cell.fill.fgColor.rgb)
                & (prev_cell.value is None)
                & (prev_cell.fill.fgColor.rgb in ["FFEDBFBB", "FFC5E0B4", "FFE88880"])
                & (prev_cell.fill.fgColor.rgb in ["FFC5E0B4", "FFE88880"])
            ) | (
                (prev_cell.fill.start_color.theme != cell.fill.start_color.theme)
                & (prev_cell.value is None)
                & (prev_cell.fill.start_color.theme == 9)
            ):
                # | (
                #     (prev_cell.fill.fgColor.tint != cell.fill.fgColor.tint)
                #     & (prev_cell.fill.fgColor.tint != 0)
                #     & (prev_cell.value is None)
                #     # & (prev_cell.fill.fgColor.rgb in ["FFC5E0B4", "FFE88880"])
                # ):
                weeks.append(list(row.keys())[col_idx - 2])
                if prev_cell.fill.fgColor.rgb in ["FFEDBFBB", '"FFE88880"']:
                    pool_changeout_types.append("I")
                else:
                    pool_changeout_types.append("P")

    return pd.Series({"weeks": weeks, "pool_changeout_type": pool_changeout_types})


# Process the dataframe
changeouts_df = (
    df.apply(lambda x: get_weeks_and_comments(x, filtered_sheet), axis=1)
    .explode(["weeks", "comments"])
    .rename(columns={"weeks": "changeout_week"})
    .pipe(idx_to_pool_slot)
)

# Extract information from comments
changeouts_df[["equipo", "component_serial"]] = changeouts_df["comments"].apply(
    lambda x: pd.Series(extract_info(x))
)

changeouts_df = changeouts_df.drop(columns=["comments"]).assign(
    changeout_date=changeouts_df["changeout_week"].map(
        lambda x: datetime.strptime(x + "-1", "%Y-W%W-%w")
    )
)
arrivals_df = (
    df.apply(lambda x: get_end_week(x, filtered_sheet), axis=1)
    .explode(["weeks", "pool_changeout_type"])
    .rename(columns={"weeks": "arrival_week"})
    .pipe(idx_to_pool_slot)
)
arrivals_df = arrivals_df.assign(
    arrival_date=arrivals_df["arrival_week"].map(
        lambda x: datetime.strptime(x + "-1", "%Y-W%W-%w")
    )
)

df = pd.merge_asof(
    changeouts_df.sort_values("changeout_date"),
    arrivals_df.sort_values("arrival_date"),
    by="pool_slot",
    left_on="changeout_date",
    right_on="arrival_date",
    direction="forward",
)  # .assign(component_code=file.split("-")[-1].split(".")[0])


changeouts_df

,pool_slot,changeout_week,equipo,component_serial,changeout_date
0,1,2023-W27,856,#EG08090003,2023-07-03
1,1,2023-W41,868,#EE13111435,2023-10-09
2,1,2024-W6,872,#EE14091124,2024-02-05
3,1,2024-W17,878,#EG07080105,2024-04-22
4,2,2023-W27,881,#EN12040115,2023-07-03
5,2,2023-W38,396,#EE14010094,2023-09-18
6,2,2023-W50,865,#EE06040239,2023-12-11
7,2,2024-W18,864,#EE14070950,2024-04-29
8,3,2023-W27,875,#EN11110073,2023-07-03
9,3,2023-W39,861,#EN12090076,2023-09-25


In [299]:
arrivals_df

,pool_slot,arrival_week,pool_changeout_type,arrival_date
0,1,2023-W40,P,2023-10-02
1,1,2024-W3,P,2024-01-15
2,1,2024-W13,P,2024-03-25
3,1,2024-W24,P,2024-06-10
4,2,2023-W34,P,2023-08-21
5,2,2023-W49,P,2023-12-04
6,2,2024-W12,P,2024-03-18
7,2,2024-W25,P,2024-06-17
8,3,2023-W38,P,2023-09-18
9,3,2024-W1,P,2024-01-01


In [289]:
sheet[4][59].fill.start_color.theme

Values must be of type <class 'int'>

In [276]:
sheet[1][57].fill.start_color.theme

Values must be of type <class 'int'>

In [272]:
sheet[1][50].fill.start_color.theme

9

In [224]:
filtered_sheet[1][40].fill.start_color.theme

0

In [200]:
filtered_sheet[1][40].fill.fgColor

<openpyxl.styles.colors.Color object>
Parameters:
rgb=None, indexed=None, auto=None, theme=0, tint=0.0, type='theme'

In [120]:
filtered_sheet[1].__len__()

131

In [118]:
filtered_sheet[1][100].fill.fgColor.rgb

Values must be of type <class 'str'>

In [105]:
filtered_sheet[1][15].fill.fgColor.rgb

'FFFF0000'

In [59]:
changeouts_df.comments.iloc[0]

'[Threaded comment]\n\nYour version of Excel allows you to read this threaded comment; however, any edits to it will get removed if the file is opened in a newer version of Excel. Learn more: https://go.microsoft.com/fwlink/?linkid=870924\n\nComment:\n    Equipo: 856\nNS: #EG08090003\nEstatus: \nReply:\n    Estatus:  En proceso de armado 12-12-23'

In [35]:
df

,componente,2023-W27,2023-W28,2023-W29,2023-W30,2023-W31,2023-W32,2023-W33,2023-W34,2023-W35,...,2025-W43,2025-W44,2025-W45,2025-W46,2025-W47,2025-W48,2025-W49,2025-W50,2025-W51,2025-W52
0,None,W27,W28,W29,W30,W31,W32,W33,W34,W35,...,W43,W44,W45,W46,W47,W48,W49,W50,W51,W52
1,N°1,1,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,N°2,1,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,N°3,1,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,N°4,None,1,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5,N°5,None,None,1,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
6,N°6,None,None,1,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
7,N°7,None,None,None,None,1,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
8,N°8,None,None,None,None,None,1,None,None,None,...,None,None,None,None,None,None,None,None,None,None
9,N°9,None,None,None,None,None,None,1,None,None,...,None,None,None,None,None,None,None,None,None,None


In [25]:
df.columns

Index([   1, None, None, None, None, None, None, None, None, None,
       ...
       None, None, None, None, None, None, None, None, None, None],
      dtype='object', length=130)

In [23]:
week_list.__len__()

131

In [18]:
df

,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
N°1,,,,,,,,,,,,,,,,,,,,,
None,W27,W28,W29,W30,W31,W32,W33,W34,W35,W36,...,W43,W44,W45,W46,W47,W48,W49,W50,W51,W52
N°1,1,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
N°2,1,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
N°3,1,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
N°4,None,1,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
N°5,None,None,1,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
N°6,None,None,1,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
N°7,None,None,None,None,1,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
N°8,None,None,None,None,None,1,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [10]:
df

,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
N°1,,,,,,,,,,,,,,,,,,,,,
None,W27,W28,W29,W30,W31,W32,W33,W34,W35,W36,...,W43,W44,W45,W46,W47,W48,W49,W50,W51,W52
N°1,1,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
N°2,1,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
N°3,1,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
N°4,None,1,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
N°5,None,None,1,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
N°6,None,None,1,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
N°7,None,None,None,None,1,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
N°8,None,None,None,None,None,1,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
